In [1]:
import numpy as np
import pandas as pd
import os
import dicom
from report_parser import parse_report
from collections import defaultdict
import pickle 
import tqdm
import sys
from time import strftime, gmtime

metrics_dir = '/scratch/wboag/2019/cxr/cxr-baselines/spring2019/scoring'
if metrics_dir not in sys.path:
    sys.path.append(metrics_dir)
    
from pycocoevalcap3.bleu.bleu   import Bleu
from pycocoevalcap3.cider.cider import Cider

/scratch/cxr-baselines/cxr-baselines/venv_cxr/lib/python3.5/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)


In [2]:
df_gt     = pd.read_csv('/scratch/wboag/2019/cxr/cxr-baselines/camera_ready/output/reference.tsv', sep='\t')
df_3gram  = pd.read_csv('/scratch/wboag/2019/cxr/cxr-baselines/camera_ready/output/3-gram.tsv'   , sep='\t')
df_knn    = pd.read_csv('/scratch/wboag/2019/cxr/cxr-baselines/camera_ready/output/knn.tsv'      , sep='\t')
df_random = pd.read_csv('/scratch/wboag/2019/cxr/cxr-baselines/camera_ready/output/random.tsv'   , sep='\t')

references  = {k:[v] for k,v in    df_gt[['dicom_id','text'     ]].values}
pred_3gram  = {k:[v] for k,v in df_3gram[['dicom_id','generated']].values}
pred_knn    = {k:[v] for k,v in   df_knn[['dicom_id','generated']].values}
pred_random = {k:[v] for k,v in df_random[['dicom_id','generated']].values}

methods = {'random':pred_random, '3gram':pred_3gram, 'knn':pred_knn}

In [3]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

for method,pred in methods.items():
    
    # Get just the predictions from this method
    ids = list(pred.keys())
    pred = {k:v for k,v in       pred.items() if (k in ids)}
    ref  = {k:v for k,v in references.items() if (k in ids)}

    bleu_scorer = Bleu(4)
    bleu_score = bleu_scorer.compute_score(ref, pred)
    print('\t%-10s (n=%6d):' % (method,len(ids)), bleu_score[0])

print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2019-11-13 16:45:56
	knn        (n= 99145): [0.28070485699305964, 0.15001069501054853, 0.09054338889074229, 0.058481876752902125]
	random     (n= 99145): [0.25361669492715455, 0.1265504006244579, 0.071727056537436, 0.043635655662199804]
	3gram      (n= 99145): [0.22583162678533808, 0.11722725423315455, 0.06874108124726297, 0.04182679357710936]
2019-11-13 17:01:38


In [4]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

for method,pred in methods.items():
    
    # Get just the predictions from this method
    ids = list(pred.keys())
    pred = {k:v for k,v in       pred.items() if (k in ids)}
    ref  = {k:v for k,v in references.items() if (k in ids)}

    cider_scorer = Cider(4)
    cider_score = cider_scorer.compute_score(ref, pred)
    print('\t%-10s (n=%6d):' % (method,len(ids)), cider_score[0])

print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2019-11-13 17:01:38
	knn        (n= 99145): 0.09213673030398213
	random     (n= 99145): 0.0461660101048
	3gram      (n= 99145): 0.018500838202221367
2019-11-13 17:24:14
